In [2]:
#Instalo dependencias si es necesario
!pip install openpyxl

In [3]:
# Importo librerías
import pandas as pd
import numpy as np


In [4]:
# Subo archivo
from google.colab import files
uploaded = files.upload()

Saving df_pronta_limpio.xlsx to df_pronta_limpio.xlsx


In [5]:
# Cargo archivo Excel
df = pd.read_excel("df_pronta_limpio.xlsx")  # Asegurate que el nombre coincida

In [6]:
# RFM Analysis
df["Fecha_Venta"] = pd.to_datetime(df["Fecha_Venta"])
fecha_referencia = df["Fecha_Venta"].max()

rfm = df.groupby("ID_Cliente").agg({
    "Fecha_Venta": lambda x: (fecha_referencia - x.max()).days,
    "ID_Venta_Int": "count",
    "Total_Venta": "sum"
}).reset_index()

rfm.columns = ["ID_Cliente", "Recency", "Frequency", "Monetary"]

rfm["R_Score"] = pd.qcut(rfm["Recency"], 4, labels=[4, 3, 2, 1]).astype(int)
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 4, labels=[1, 2, 3, 4]).astype(int)
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 4, labels=[1, 2, 3, 4]).astype(int)
rfm["RFM_Score"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)

In [7]:
# ABC Analysis
ventas_productos = df.groupby("Producto")["Total_Venta"].sum().reset_index()
ventas_productos = ventas_productos.sort_values(by="Total_Venta", ascending=False)
ventas_productos["Porcentaje"] = ventas_productos["Total_Venta"] / ventas_productos["Total_Venta"].sum()
ventas_productos["Porcentaje_Acumulado"] = ventas_productos["Porcentaje"].cumsum()

def clasificar_abc(p):
    if p <= 0.80:
        return "A"
    elif p <= 0.95:
        return "B"
    else:
        return "C"

ventas_productos["Clasificación_ABC"] = ventas_productos["Porcentaje_Acumulado"].apply(clasificar_abc)

In [8]:
# Guardo ambos análisis en un Excel
with pd.ExcelWriter("analisis_rfm_abc.xlsx", engine="openpyxl") as writer:
    rfm.to_excel(writer, sheet_name="RFM_Clientes", index=False)
    ventas_productos.to_excel(writer, sheet_name="ABC_Productos", index=False)

In [9]:
# Descargo el archivo
files.download("analisis_rfm_abc.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>